In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import scipy.signal as sps
# %matplotlib notebook
%matplotlib inline

The sampling rate is 1kHz. \
reference: https://www.nature.com/articles/sdata201820

In [ ]:
df_2_1 = pd.read_csv('../data/FINGERTIP_PPG_FROM_HYPERTENSIVE_SUBJECTS/0_subject/2_1.txt', sep='\t', header=None)

In [ ]:
df_2_1 = df_2_1.T

In [ ]:
df_2_1.head()

In [ ]:
plt.plot(df_2_1)
plt.show()

In [ ]:
raw_signal = df_2_1.values.T[0]

In [ ]:
raw_signal = raw_signal[:-1]

In [ ]:
raw_signal

In [ ]:
# using FFT to extract the wave frequency of raw signal 
from scipy import fftpack
f_s=1000

X = fftpack.fft(raw_signal)
freqs = fftpack.fftfreq(len(raw_signal)) * f_s

fig, ax = plt.subplots()

ax.stem(freqs, np.abs(X))
ax.set_title('frequency domain_PPGBP_0subject_2_1')
ax.set_xlabel('Frequency in Hertz [Hz]')
ax.set_ylabel('Frequency Domain (Spectrum) Magnitude')
ax.set_xlim(-20, 20)
ax.set_ylim(-5, 1e5)
plt.tight_layout()
plt.savefig('frequency domain_PPGBP_0subject_2_1.png', dpi=150)

In [ ]:
np.abs(X)

## convolve & guassian

In [ ]:
win_size = int(50)

In [ ]:
moving_ave_win = np.ones(win_size) / win_size

In [ ]:
filter_signal = np.convolve(signal,1000, "same")

In [ ]:
plt.plot(filter_signal)
plt.show()

In [ ]:
lowpass_gaussian_order=3
lowpass_gaussian_std=0.8

In [ ]:
gauss_filt_win = sps.gaussian(lowpass_gaussian_order, lowpass_gaussian_std)
gauss_filt_win = gauss_filt_win / np.sum(gauss_filt_win)
pre_proc_signal = np.convolve(signal, gauss_filt_win, "same")

In [ ]:
plt.plot(pre_proc_signal)
plt.show()

## test butter filter

In [ ]:
b, a = sps.butter(4, 100, 'low', analog=True)
w, h = sps.freqs(b, a)
plt.semilogx(w, 20 * np.log10(abs(h)))
plt.title('Butterworth filter frequency response')
plt.xlabel('Frequency [radians / second]')
plt.ylabel('Amplitude [dB]')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')
plt.axvline(100, color='green') # cutoff frequency
plt.show()

In [ ]:
t = np.linspace(0, 1, 1000, False)  # 1 second
sig = np.sin(2*np.pi*10*t) + np.sin(2*np.pi*20*t)
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.plot(t, sig)
ax1.set_title('10 Hz and 20 Hz sinusoids')
ax1.axis([0, 1, -2, 2])

sos = sps.butter(10, 15, 'hp', fs=1000, output='sos')
filtered = sps.sosfilt(sos, sig)
ax2.plot(t, filtered)
ax2.set_title('After 15 Hz high-pass filter')
ax2.axis([0, 1, -2, 2])
ax2.set_xlabel('Time [seconds]')
plt.tight_layout()
plt.show()

In [ ]:
t = np.linspace(0, 1, 1000, False)  # 1 second
# f1 = 10Hz, f2 = 20Hz 
sig = np.sin(2*np.pi*10*t) + np.sin(2*np.pi*20*t)
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.plot(t, sig)
ax1.set_title('10 Hz and 20 Hz sinusoids')
ax1.axis([0, 1, -2, 2])

# if no fs=1000, we need to figure the Wn = Fc/Fs * 2 = (15/1000) * 2 = 0.03
sos = sps.butter(10, 0.015*2, 'hp', output='sos')
filtered = sps.sosfilt(sos, sig)
ax2.plot(t, filtered)
ax2.set_title('After 15 Hz high-pass filter')
ax2.axis([0, 1, -2, 2])
ax2.set_xlabel('Time [seconds]')
plt.tight_layout()
plt.show()

In [ ]:
# using FFT to extract the wave frequency of raw signal 
from scipy import fftpack
f_s=1000

X = fftpack.fft(sig)
freqs = fftpack.fftfreq(len(sig)) * f_s

fig, ax = plt.subplots()

ax.stem(freqs, np.abs(X))
ax.set_xlabel('Frequency in Hertz [Hz]')
ax.set_ylabel('Frequency Domain (Spectrum) Magnitude')
ax.set_xlim(-50, 50)
ax.set_ylim(-5, 110)

Here we can see that the f1 = 20 Hz and f2 = 10 Hz are extracted. 

In [ ]:
f0=0.1
b, a = sps.butter(4, 0.1*2)
sos = sps.butter(8, 0.1*2, output='sos')

w, h = sps.freqz(b, a)
# print(w)
plt.plot(w/np.pi/2, 20*np.log10(np.abs(h)))
plt.show()

In [ ]:
print(a, b)
print(sos.shape)
print(sos[0].shape)
# print(sos[1].shape)

In [ ]:
sos = sps.butter(10, 15, 'hp', fs=1000, output='sos')
filtered = sps.sosfilt(sos, raw_signal)

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.plot(raw_signal)
ax1.set_title('raw signal with 1kHz sampling rate')

ax2.plot(filtered)
ax2.set_title('after 15 Hz high-pass filter')
plt.show()

In [ ]:
sos = sps.butter(10, 0.03, 'hp', output='sos')
filtered = sps.sosfilt(sos, raw_signal)

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.plot(raw_signal)
ax1.set_title('raw signal with 1kHz sampling rate')

ax2.plot(filtered)
ax2.set_title('after 15 Hz high-pass filter')
plt.show()

In [ ]:
fc = 10
sos = sps.butter(6, fc, 'lp', fs=1000, output='sos')
filtered = sps.sosfilt(sos, raw_signal)

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.plot(raw_signal)
ax1.set_title('raw signal with 1kHz sampling rate')

ax2.plot(raw_signal, label='raw signal')
ax2.plot(filtered, label=f'butter(m=6, cutoff={fc}Hz)')
ax2.set_title(f'after {fc} Hz Butter low-pass filter and Butter filter')
ax2.legend(bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
sos_order6 = sps.butter(6, 0.03, 'lp', output='sos')
filtered_order6 = sps.sosfilt(sos_order6, raw_signal)

sos_order4 = sps.butter(4, 0.03, 'lp', output='sos')
filtered_order4 = sps.sosfilt(sos_order4, raw_signal)

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.plot(raw_signal)
ax1.set_title('raw signal with 1kHz sampling rate')

ax2.plot(raw_signal, label='raw signal')
ax2.plot(filtered_order6, label='butter(m=6, cutoff=15Hz)')
ax2.plot(filtered_order4, label='butter(m=4, cutoff=15Hz)')
ax2.set_title('after 15 Hz low-pass filter')
ax2.set_title('after 15 Hz Butter low-pass filter and Butter filter')
ax2.legend(bbox_to_anchor=(1, 0.5))
plt.show()

## test chebyshev1 filter 

In [ ]:
b, a = sps.cheby1(4, 5, 100, 'low', analog=True)
w, h = sps.freqs(b, a)
plt.semilogx(w, 20 * np.log10(abs(h)))
plt.title('Chebyshev Type I frequency response (rp=5)')
plt.xlabel('Frequency [radians / second]')
plt.ylabel('Amplitude [dB]')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')
plt.axvline(100, color='green') # cutoff frequency
plt.axhline(-5, color='green') # rp
plt.show()

In [ ]:
sos = sps.cheby1(3, 1, 15, 'lp', fs=1000, output='sos')
filtered_cheb1 = sps.sosfilt(sos, raw_signal)

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.plot(raw_signal)
ax1.set_title('raw signal with 1kHz sampling rate')

ax2.plot(raw_signal, label='raw signal')
ax2.plot(filtered_cheb1, label='cheb1(m=3, Rp=1, cutoff=15Hz)')
ax2.plot(filtered, label='butter(m=6, cutoff=15Hz)')
ax2.set_title('after 15 Hz Butter low-pass filter and Chebyshev1 filter')
ax2.legend(bbox_to_anchor=(1, 0.5))
plt.show()

## test chebyshev2 filter

In [ ]:
b, a = sps.cheby2(4, 40, 100, 'low', analog=True)
w, h = sps.freqs(b, a)
plt.semilogx(w, 20 * np.log10(abs(h)))
plt.title('Chebyshev Type II frequency response (rs=40)')
plt.xlabel('Frequency [radians / second]')
plt.ylabel('Amplitude [dB]')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')
plt.axvline(100, color='green') # cutoff frequency
plt.axhline(-40, color='green') # rs
plt.show()

In [ ]:
b, a = sps.cheby2(4, 40, 100, 'high', analog=True)
w, h = sps.freqs(b, a)
plt.semilogx(w, 20 * np.log10(abs(h)))
plt.title('Chebyshev Type II frequency response (rs=40)')
plt.xlabel('Frequency [radians / second]')
plt.ylabel('Amplitude [dB]')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')
plt.axvline(100, color='green') # cutoff frequency
plt.axhline(-40, color='green') # rs
plt.show()

In [ ]:
Wn = [500/750, 560/750]
sos = sps.cheby2(10, 40, Wn, btype='bandpass', output='sos')

b, a = sps.cheby2(10, 40, Wn, btype='bandpass', analog=True)
w, h = sps.freqs(b, a)
# plt.semilogx(w, 20 * np.log10(abs(h)))
plt.plot(w*750, 20 * np.log10(abs(h)))
plt.title('Chebyshev Type II frequency response (rs=40)')
plt.xlabel('Frequency [radians / second]')
plt.ylabel('Amplitude [dB]')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')
plt.axvline(100, color='green') # cutoff frequency
plt.axhline(-40, color='green') # rs
plt.xlim([0, 750])
plt.show()

In [ ]:
w*750

In [ ]:
sos

In [ ]:
sos = sps.cheby2(6, 18, 17, 'lp', fs=1000, output='sos')
filtered_cheb2 = sps.sosfilt(sos, raw_signal)

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.plot(raw_signal)
ax1.set_title('raw signal with 1kHz sampling rate')

ax2.plot(raw_signal, label='raw signal')
ax2.plot(filtered_cheb2, label='cheb2(m=6, Rs=18, cutoff=17Hz)')
ax2.plot(filtered, label='butter(m=6, cutoff=15Hz)')
ax2.set_title('after 17 Hz Butter low-pass filter and Chebyshev2 filter')
ax2.legend(bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
%matplotlib notebook
sos_order6 = sps.cheby2(6, 20, 0.03, 'lp', output='sos')
filtered_cheb2_order6 = sps.sosfilt(sos_order6, raw_signal)

sos_order4 = sps.cheby2(4, 20, 0.03, 'lp', output='sos')
filtered_cheb2_order4 = sps.sosfilt(sos_order4, raw_signal)

sos_butter_order4 = sps.butter(4, 0.03, 'lp', output='sos')
filtered_butter_order4 = sps.sosfilt(sos_butter_order4, raw_signal)

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.plot(raw_signal)
ax1.plot(filtered_butter_order4, label='butter(m=4, cutoff=15Hz)')
ax1.set_title('raw signal with 1kHz sampling rate')

ax2.plot(raw_signal, label='raw signal')
ax2.plot(filtered_cheb2_order6, label='cheb2(m=6, Rs=20, cutoff=15Hz)')
ax2.plot(filtered_cheb2_order4, label='cheb2(m=4, Rs=20, cutoff=15Hz)')
ax2.set_title('after 15Hz Chebyshev2 low-pass filter: n=4 vs n=6')
ax2.legend(bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
sos_order4

In [ ]:
sos = sps.cheby2(6, 18, 17, 'hp', fs=1000, output='sos')
filtered_cheb2_highpass = sps.sosfilt(sos, raw_signal)

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.plot(raw_signal)
ax1.set_title('raw signal with 1kHz sampling rate')

ax2.plot(raw_signal, label='raw signal')
ax2.plot(filtered_cheb2_highpass, label='cheb2(m=6, Rs=18, cutoff=17Hz)')
ax2.set_title('after 17 Hz Chebyshev2 high-pass filter')
plt.show()

## test elliptic filter

In [ ]:
b, a = sps.ellip(4, 5, 40, 100, 'low', analog=True)
w, h = sps.freqs(b, a)
plt.semilogx(w, 20 * np.log10(abs(h)))
plt.title('Elliptic filter frequency response (rp=5, rs=40)')
plt.xlabel('Frequency [radians / second]')
plt.ylabel('Amplitude [dB]')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')
plt.axvline(100, color='green') # cutoff frequency
plt.axhline(-40, color='green') # rs
plt.axhline(-5, color='green') # rp
plt.show()

In [ ]:
sos = sps.ellip(3, 1, 18, 17, 'lp', fs=1000, output='sos')
filtered_ellip = sps.sosfilt(sos, raw_signal)

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.plot(raw_signal)
ax1.set_title('raw signal with 1kHz sampling rate')

ax2.plot(raw_signal, label='raw signal')
ax2.plot(filtered_cheb2, label='cheb2(m=6, Rs=18, cutoff=17Hz)')
ax2.plot(filtered, label='butter(m=6, cutoff=15Hz)')
ax2.plot(filtered_ellip, label='elliptic(m=3, Rp=1, Rs=18, cutoff=17Hz)')
ax2.set_title('after 17 Hz elliptic low-pass filter filter')
ax2.legend(bbox_to_anchor=(1, 0.5))
plt.show()

## Median filter

In [ ]:
fs = 25
window_size = 50
nums = (1/window_size) * np.ones(window_size)

w1, h1 = sps.freqz(nums, fs=fs)

# plt.semilogx(w, 20 * np.log10(abs(h)))
plt.semilogx(w1, abs(h1))
plt.title('Mean filter with window size of 50 frequency response')
plt.xlabel('Frequency [Hz]')
# plt.ylabel('Amplitude [dB]')
plt.ylabel('Gain')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')
# plt.axvline(100, color='green') # cutoff frequency
# plt.axhline(-40, color='green') # rs

plt.savefig('mean filter with window size of 50.png', dpi=250)
plt.show()

In [ ]:
len(w1)

In [ ]:
w1[abs(h1) < 0.5]

## Gaussian filter 

In [ ]:
lowpass_gaussian_order=3
lowpass_gaussian_std=0.8
gauss_filt_win = sps.gaussian(lowpass_gaussian_order, lowpass_gaussian_std)
gauss_filt_win = gauss_filt_win / np.sum(gauss_filt_win)

In [ ]:
w2, h2 = sps.freqz(gauss_filt_win, fs=25)

plt.semilogx(w2, abs(h2))
plt.title('Gaussian filter with order=3, std=0.8 frequency response')
plt.xlabel('Frequency [Hz]')
# plt.ylabel('Amplitude [dB]')
plt.ylabel('Gain')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')
# plt.axvline(100, color='green') # cutoff frequency
# plt.axhline(-40, color='green') # rs

plt.savefig('Gaussian filter with order=3 std=0.8.png', dpi=250)
plt.show()

In [ ]:
w2[abs(h2) < 0.5]

In [ ]:
plt.semilogx(w1, 1-abs(h1))
plt.semilogx(w2, abs(h2))
plt.title('Mean filter and Gaussian filter frequency response')
plt.xlabel('Frequency [Hz]')
# plt.ylabel('Amplitude [dB]')
plt.ylabel('Gain')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')
# plt.axvline(100, color='green') # cutoff frequency
# plt.axhline(-40, color='green') # rs

plt.savefig('Mean filter and Gaussian filter.png', dpi=250)
plt.show()

In [ ]:
w2

## FIR filter (Finite impulse response filter)

## wavelet denoising filter